In [1]:
import sys
sys.path.insert(1, '/work/IFT6010_Story_Tree/src/features/')

from utils_cosine_tf_idf import get_corpus_tf_idf
from utils_cosine_tf_idf import do_cosine_similarity
import utils_cosine_tf_idf as util

import pandas as pd
import pickle
import nltk

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Clusters events with community

**Get dataset**

In [2]:
news_dataset = pd.read_pickle("/work/IFT6010_Story_Tree/data/short_news_dataset_2_with_extractedkeyword.pickle").drop_duplicates(subset=['TEXT'])

**Get communities**

In [ ]:
temp_defaultdict_communities = pickle.load(open("extracted_communities.pickle",'rb'))
dict_communities = {}
for i,v in temp_defaultdict_communities.items():
    if len(v)==1:
        dict_communities[i] = v[0]

In [ ]:
keywords_clusters = pd.DataFrame(dict_communities.items())
keywords_clusters.columns = ['keyword', 'cluster']

**Get tfidf vector for keywords**

In [ ]:
#arg 1 = dataframe source
#arg 2 = colomne d'ou le texte provient (je l'ai mis en argument parce que je ne connais pas la structure des autres dataframe)
##arg 3 = ngram a considerer... possible solution pour les keywords a n mots (n>1)

dataset_tf_idf, dict_weights = get_corpus_tf_idf(news_dataset,'TEXT',ngram=1)
dataset_tf_idf = dataset_tf_idf.drop_duplicates(subset=['TEXT'])

In [ ]:
from nltk.stem import PorterStemmer
porter = PorterStemmer()
lemmatizer = porter.stem

dict_weights_lemmatized = {}
for i,v in dict_weights.items():
    dict_weights_lemmatized[lemmatizer(i)] = v

In [ ]:
do_cosine_similarity(dataset_tf_idf.iloc[0]['VECTOR'],dataset_tf_idf.iloc[2]['VECTOR'])

0.25776752962351324

**Assign document to community**

In [ ]:
# for the cluster, we assume they are in a dataframe
def prepare_cluster_vector(dataframe, dict_weights):
    all_cluster = dataframe['cluster'].unique()

    all_cluster_dict = {}

    for cluster in all_cluster:
        cluster_dict ={}
        all_keyword=list(dataframe[dataframe["cluster"] == cluster]['keyword'])
        all_keyword=' '.join(all_keyword)

        # We will have more keywords than 
        tokens = nltk.word_tokenize(all_keyword)
        
        #!!!!DEAL with keyword with multiple word ... some keyword are not int the dictionary of all weights
        for token in tokens:
            if token in dict_weights:
                cluster_dict[token]=dict_weights[token]
            else:
                cluster_dict[token]=0

        # print(cluster_dict)
        all_cluster_dict[cluster]=cluster_dict
        
    return all_cluster_dict

In [ ]:
clusters_vector = prepare_cluster_vector(keywords_clusters, dict_weights_lemmatized)

In [ ]:
total_len=0
for i in range(len(clusters_vector)):
    
    total_len += len(clusters_vector[i])
print(total_len)

761


In [ ]:
do_cosine_similarity(dataset_tf_idf.iloc[2]['VECTOR'],clusters_vector[0])

0.017989580910794833

In [ ]:
def set_document_topic(document_vector, all_cluster_vector):
    max_cluster_name=''
    max_cluster_value=0
    for cluster in all_cluster_vector:
        cluster_value= do_cosine_similarity(document_vector,clusters_vector[cluster])
        # print(cluster_value)

        if (max_cluster_value<cluster_value):
            max_cluster_value=cluster_value
            max_cluster_name=cluster
    
    # print(max_cluster_name)
    # print(max_cluster_value)
    return max_cluster_name

In [ ]:
dataset_tf_idf['TOPIC']=dataset_tf_idf['VECTOR'].apply(set_document_topic,all_cluster_vector=clusters_vector)

In [ ]:
dataset_tf_idf['TOPIC'].value_counts()

0    488
1    270
2     36
3     22
5      5
       4
4      2
Name: TOPIC, dtype: int64

In [ ]:
dataset_tf_idf[dataset_tf_idf['TOPIC'] == 1]

,Unnamed: 0,KEYWORDS,SUMMARY,TEXT,TITLE,date,extracted_keywords,PREPROCESSING_TEXT,VECTOR,TOPIC
1,1,"['facebook', 'whats', 'world', 'unfolds', 'tam...",Chat with us in Facebook Messenger.\nFind out ...,Chat with us in Facebook Messenger. Find out w...,Tamagotchi is back,10-06-2017,"[messenger, facebook, chat, find, unfolds, hap...",chat us facebook messenger find 's happening w...,"{'chat': 0.0011021343105938432, 'us': 0.015378...",1
2,2,"['jedi', 'shots', 'rey', 'force', 'wars', 'sta...","ESPN's ""Monday Night Football"" had bears, viki...","ESPN's ""Monday Night Football"" had bears, viki...",'Star Wars: The Last Jedi' trailer debuts on '...,10-06-2017,"[monday, trailer, night, rey, jedi, espn, foot...",espn 's monday night football '' bears vikings...,"{'espn': 0.0024102761351745593, 'monday': 0.00...",1
8,8,"['look', 'response', 'force', 'trump', 'tour',...",(CNN) Finally lumbering into a devastated Puer...,"Michael D'Antonio is the author of the book "" ...",Trump in Puerto Rico: A narcissist's tour de f...,10-06-2017,"[trump, puerto, reality, success, press, pursu...",michael d'antonio author book never enough don...,"{'michael': 0.002286875035651508, 'author': 0....",1
9,9,"['okunoin', 'tohoku', 'japan', 'risshakuji', '...","(CNN) — Upon hearing I would have to climb 1,0...",\n\n\n\nThis article was first published in Ju...,"Yamadera Risshakuji in Tohoku: 1,015 steps to ...",10-06-2017,"[temple, cnn, risshakuji, japan, hall, june, b...",article first published june editor 's note cn...,"{'article': 0.0017379592770653371, 'first': 0....",1
19,19,"['according', 'boundaries', 'generations', 'ac...","""The only generation we do define is Baby Boom...",We can all agree that Millennials are the wors...,"Here Is When Each Generation Begins and Ends, ...",10-06-2017,"[millennials, slate, diprete, article, worst, ...",agree millennials worst millennial fight new y...,"{'agree': 0.0017977724690039125, 'millennials'...",1
...,...,...,...,...,...,...,...,...,...,...
61,61,"['america', 'peter', 'tend', 'prosperous', 'ur...","However, the Democrats often behave as if they...",Editor's note: The following column originally...,Peter Morici: Trump’s immigration proposals wo...,10-06-2017,"[trump, baltimore, immigrants, growth, border,...",editor 's note following column originally app...,"{'editor': 0.0014681382248323404, 'note': 0.00...",1
64,64,"['launch', 'hellcat', 'challenger', 'car', 'tr...",There’s no better place to test drive the Dodg...,There’s no better place to test drive the Dodg...,2018 Dodge Challenger SRT Demon test drive,10-06-2017,"[challenger, srt, launch, control, dodge, engi...",better place test drive dodge challenger srt d...,"{'better': 0.00556479560337112, 'place': 0.005...",1
67,67,"['youre', 'answer', 'halloween', 'moments', 'r...",What's the difference between a Halloween cost...,What's the difference between a Halloween cost...,Cosplay or Halloween costume? A cosplay expert...,10-06-2017,"[comic, halloween, voltron, orsini, asuncion, ...",'s difference halloween costume cosplay 're co...,"{'difference': 0.0015408083485677227, 'hallowe...",1
69,69,"['pizzas', 'pizza', 'hut', 'degrees', 'hotter'...",Pizza Hut is introducing a new triple-insulate...,Pizza Hut is introducing a new triple-insulate...,Pizza Hut has new pouches that deliver pizzas ...,10-06-2017,"[pouch, pizzas, allen, parka, crisp, oct, pdt,...",pizza hut introducing new tripleinsulated deli...,"{'pizza': 0.001057192793129747, 'hut': 0.00046...",1


# Regroup document that talk about the same event

In [14]:
!pip install gensim

     |████████████████████████████████| 23.9 MB 28.1 MB/s 
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [23]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import re

In [4]:
#news_dataset

In [10]:
len(news_dataset)

827

In [28]:
documents = [TaggedDocument(doc["TEXT"], [i]) for i, doc in news_dataset.iterrows()]
model = Doc2Vec(documents, vector_size=150, window=3, min_count=1, workers=4)

In [29]:
vector = model.infer_vector(re.split('"|\n| ', news_dataset.iloc[1]["TEXT"]))

In [27]:
vector

array([ 0.05076697, -0.39840895, -0.24151756, -0.3049555 , -0.06484894],
      dtype=float32)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=902703f2-430f-48f3-ba3f-6c2fee66cf11' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>